<h2> Imports </h2>

In [57]:
import pandas as pd
import numpy as np
import json
import os
import math

<h1> Step 1: Prepare data and configure the environment </h1>

<h3> 1. Data Loading (Data I/O) </h3>

In [58]:
# Define the base directories
dataset_dir = '../Data_Generator/datasets'
topology_dir = '../Topology_Generator/topologies'

Loading dataset

In [59]:
# Initialize dictionaries for datasets and topologies
datasets = {}

# Function to load datasets from a directory
def load_datasets_from_directory(dataset_dir):
    
    # Loop through each folder in the datasets directory
    for dataset_name in os.listdir(dataset_dir):
        dataset_path = os.path.join(dataset_dir, dataset_name)
        
        # Only process directories
        if os.path.isdir(dataset_path):
            episodes = os.path.join(dataset_path, "episodes.csv")
            agents = os.path.join(dataset_path, "agents.csv")
            arrivals = os.path.join(dataset_path, "arrivals.csv")
            tasks = os.path.join(dataset_path, "tasks.csv")
            
            dataset = {}
            dataset['episodes'] = pd.read_csv(episodes)
            dataset['agents'] = pd.read_csv(agents)
            dataset['arrivals'] = pd.read_csv(arrivals)
            dataset['tasks'] = pd.read_csv(tasks)
            datasets[dataset_name] = dataset


In [60]:
load_datasets_from_directory(dataset_dir)

print("dataset heavy -> agents")
display(datasets['heavy']['agents'].head())
datasets['heavy']['agents'].info()

print("\ndataset heavy -> arrivals")
display(datasets['heavy']['arrivals'].head())
datasets['heavy']['arrivals'].info()

print("\ndataset heavy -> episodes")
display(datasets['heavy']['episodes'].head())
datasets['heavy']['episodes'].info()

print("\ndataset heavy -> tasks")
display(datasets['heavy']['tasks'].head())
datasets['heavy']['tasks'].info()

dataset heavy -> agents


,agent_id,f_local,m_local,lam_sec
0,0,1.741183e+09,5.713850e+09,0.708673
1,1,1.352326e+09,4.566429e+09,0.234989
2,2,1.726668e+09,5.815120e+09,0.228174
3,3,1.543616e+09,3.539850e+09,0.310369
4,4,1.130883e+09,4.161368e+09,0.548990


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   agent_id  18 non-null     int64  
 1   f_local   18 non-null     float64
 2   m_local   18 non-null     float64
 3   lam_sec   18 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 704.0 bytes

dataset heavy -> arrivals


,scenario,episode_id,t_slot,t_time,agent_id,task_id
0,heavy,0,0,0.0,0,0
1,heavy,0,0,0.0,1,1
2,heavy,0,0,0.0,4,2
3,heavy,0,0,0.0,7,3
4,heavy,0,0,0.0,10,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30636 entries, 0 to 30635
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   scenario    30636 non-null  object 
 1   episode_id  30636 non-null  int64  
 2   t_slot      30636 non-null  int64  
 3   t_time      30636 non-null  float64
 4   agent_id    30636 non-null  int64  
 5   task_id     30636 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 1.4+ MB

dataset heavy -> episodes


,scenario,episode_id,Delta,T_slots,hours,N_agents,seed
0,heavy,0,1.0,3600,1.0,18,345


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   scenario    1 non-null      object 
 1   episode_id  1 non-null      int64  
 2   Delta       1 non-null      float64
 3   T_slots     1 non-null      int64  
 4   hours       1 non-null      float64
 5   N_agents    1 non-null      int64  
 6   seed        1 non-null      int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 184.0+ bytes

dataset heavy -> tasks


,scenario,episode_id,task_id,agent_id,t_arrival_slot,t_arrival_time,b_mb,rho_cyc_per_mb,c_cycles,mem_mb,modality,has_deadline,deadline_s,deadline_time,non_atomic,split_ratio,action_space_hint
0,heavy,0,0,0,0,0.0,5.000003,1.499999e+09,7.499998e+09,64.000006,sensor,1,0.800726,0.800726,0,0.000000,discrete
1,heavy,0,1,1,0,0.0,5.000001,1.500000e+09,7.499999e+09,64.000031,image,1,0.615113,0.615113,0,0.000000,discrete
2,heavy,0,2,4,0,0.0,5.000005,1.500002e+09,7.500016e+09,64.000021,text,1,0.323007,0.323007,1,0.539704,continuous
3,heavy,0,3,7,0,0.0,5.000002,1.500001e+09,7.500006e+09,63.999980,sensor,1,0.481587,0.481587,0,0.000000,discrete
4,heavy,0,4,10,0,0.0,5.000008,1.499999e+09,7.500006e+09,64.000022,sensor,1,0.594564,0.594564,0,0.000000,discrete


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30636 entries, 0 to 30635
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   scenario           30636 non-null  object 
 1   episode_id         30636 non-null  int64  
 2   task_id            30636 non-null  int64  
 3   agent_id           30636 non-null  int64  
 4   t_arrival_slot     30636 non-null  int64  
 5   t_arrival_time     30636 non-null  float64
 6   b_mb               30636 non-null  float64
 7   rho_cyc_per_mb     30636 non-null  float64
 8   c_cycles           30636 non-null  float64
 9   mem_mb             30636 non-null  float64
 10  modality           30636 non-null  object 
 11  has_deadline       30636 non-null  int64  
 12  deadline_s         10673 non-null  float64
 13  deadline_time      10673 non-null  float64
 14  non_atomic         30636 non-null  int64  
 15  split_ratio        30636 non-null  float64
 16  action_space_hint  306

Loading topology

In [61]:
topologies = {}

def load_topologies_from_directory(topology_dir):
    
    for topology_name in os.listdir(topology_dir):
        topology_path = os.path.join(topology_dir, topology_name)
        
        # Only process directories
        if os.path.isdir(topology_path):
            topology_json_path = os.path.join(topology_path, "topology.json")
            meta_json_path = os.path.join(topology_path, "topology_meta.json")
            connection_matrix_csv_path = os.path.join(topology_path, "connection_matrix.csv")
            
             # --- Load JSON & CSV files ---
            topology_data = None
            meta_data = None
            with open(topology_json_path, "r", encoding="utf-8") as f:
                topology_data = json.load(f)
            with open(meta_json_path, "r", encoding="utf-8") as f:
                meta_data = json.load(f)
            
            # The first column is just for displaying row names, not part of the capacity matrix. 
            # So the best way is to index the first column. (index_col=0)
            connection_matrix = pd.read_csv(connection_matrix_csv_path, index_col=0)
            
            # Store the topology details and the loaded CSV
            topologies[topology_name] = {
                "topology_data": topology_data,
                "meta_data": meta_data,
                "connection_matrix": connection_matrix  # Store the loaded CSV data
            }


In [62]:
load_topologies_from_directory(topology_dir)

print('topology clustered -> connection_matrix')
display(topologies['clustered']['connection_matrix'].head())
topologies['clustered']['connection_matrix'].info()

print('\ntopology clustered -> topology_data')
print(topologies['clustered']['topology_data'])

print('\ntopology clustered -> meta_data')
print(topologies['clustered']['meta_data'])


topology clustered -> connection_matrix


,mec_0,mec_1,mec_2,mec_3,mec_4,mec_5,mec_6,mec_7,mec_8,mec_9,mec_10,mec_11,mec_12,mec_13,mec_14,mec_15,mec_16,mec_17,cloud
mec_0,0.000000,9.798801,11.958295,11.470404,9.980195,8.814050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.912800
mec_1,9.798801,0.000000,8.375143,11.535600,10.702584,9.136893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.609202
mec_2,11.958295,8.375143,0.000000,8.814632,9.637714,9.170700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92.055348
mec_3,11.470404,11.535600,8.814632,0.000000,10.377054,11.920252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.851664
mec_4,9.980195,10.702584,9.637714,10.377054,0.000000,8.105383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.076301


<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, mec_0 to mec_17
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mec_0   18 non-null     float64
 1   mec_1   18 non-null     float64
 2   mec_2   18 non-null     float64
 3   mec_3   18 non-null     float64
 4   mec_4   18 non-null     float64
 5   mec_5   18 non-null     float64
 6   mec_6   18 non-null     float64
 7   mec_7   18 non-null     float64
 8   mec_8   18 non-null     float64
 9   mec_9   18 non-null     float64
 10  mec_10  18 non-null     float64
 11  mec_11  18 non-null     float64
 12  mec_12  18 non-null     float64
 13  mec_13  18 non-null     float64
 14  mec_14  18 non-null     float64
 15  mec_15  18 non-null     float64
 16  mec_16  18 non-null     float64
 17  mec_17  18 non-null     float64
 18  cloud   18 non-null     float64
dtypes: float64(19)
memory usage: 2.8+ KB

topology clustered -> topology_data
{'number_of_servers': 18, 'private_cpu

<h3> 2. Data Validation </h3>

Before using the data, we must validate that required columns exist and that IDs match properly.

**The code below performs three layers of checks:** 

- Validate each dataset (episodes/agents/arrivals/tasks)

- Validate each topology (JSON and connection matrix)

- Validate dataset–topology pairs for unit alignment and overall consistency

In [63]:
# ---------- Generic helpers ----------
def _require(cond: bool, msg: str, errors: list):
    # Collect errors instead of stopping at first failure
    if not cond:
        errors.append(msg)

def _has_cols(df: pd.DataFrame, cols: list) -> bool:
    return all(c in df.columns for c in cols)

# ---------- Dataset-level validation ----------
def validate_one_dataset(dataset_name: str, ds: dict) -> list:
    """
    Validate a single dataset scenario (light/moderate/heavy).
    Expects keys: episodes, agents, arrivals, tasks  (each is a DataFrame)
    """
    errors = []
    episodes = ds.get("episodes")
    agents   = ds.get("agents")
    arrivals = ds.get("arrivals")
    tasks    = ds.get("tasks")

    # 1) Presence checks
    _require(isinstance(episodes, pd.DataFrame), f"[{dataset_name}] episodes missing or not a DataFrame", errors)
    _require(isinstance(agents,   pd.DataFrame), f"[{dataset_name}] agents missing or not a DataFrame", errors)
    _require(isinstance(arrivals, pd.DataFrame), f"[{dataset_name}] arrivals missing or not a DataFrame", errors)
    _require(isinstance(tasks,    pd.DataFrame), f"[{dataset_name}] tasks missing or not a DataFrame", errors)

    if errors:
        return errors  # can't continue without frames

    # 2) Required columns based on your generator outputs
    req_ep_cols  = ["scenario", "episode_id", "Delta", "T_slots", "hours", "N_agents", "seed"]
    req_ag_cols  = ["agent_id", "f_local", "m_local", "lam_sec"]
    req_ar_cols  = ["scenario", "episode_id", "t_slot", "t_time", "agent_id", "task_id"]
    req_tk_cols  = [
        "scenario","episode_id","task_id","agent_id","t_arrival_slot","t_arrival_time",
        "b_mb","rho_cyc_per_mb","c_cycles","mem_mb","modality",
        "has_deadline","deadline_s","deadline_time","non_atomic","split_ratio","action_space_hint"
    ]

    _require(_has_cols(episodes, req_ep_cols), f"[{dataset_name}] episodes missing required columns", errors)
    _require(_has_cols(agents,   req_ag_cols), f"[{dataset_name}] agents missing required columns", errors)
    _require(_has_cols(arrivals, req_ar_cols), f"[{dataset_name}] arrivals missing required columns", errors)
    _require(_has_cols(tasks,    req_tk_cols), f"[{dataset_name}] tasks missing required columns", errors)

    if errors:
        return errors

    # 3) Basic integrity checks
    # 3.1 Unique task_id
    _require(tasks["task_id"].is_unique, f"[{dataset_name}] task_id is not unique", errors)

    # 3.2 Agent id range and count
    if len(agents):
        min_id = agents["agent_id"].min()
        max_id = agents["agent_id"].max()
        expected_n = episodes["N_agents"].iloc[0]
        _require(min_id == 0, f"[{dataset_name}] agent_id should start at 0 (got {min_id})", errors)
        _require(max_id == expected_n - 1, f"[{dataset_name}] agent_id max should be N_agents-1 ({expected_n-1}), got {max_id}", errors)

    # 3.3 Cross references (arrivals/tasks refer to valid agents)
    valid_agents = set(agents["agent_id"].tolist())
    bad_arr_agents = set(arrivals["agent_id"]) - valid_agents
    bad_task_agents = set(tasks["agent_id"]) - valid_agents
    _require(len(bad_arr_agents) == 0, f"[{dataset_name}] arrivals contain unknown agent_id(s): {sorted(bad_arr_agents)}", errors)
    _require(len(bad_task_agents) == 0, f"[{dataset_name}] tasks contain unknown agent_id(s): {sorted(bad_task_agents)}", errors)

    # 3.4 Non-negative and sane numeric values
    for col in ["b_mb","rho_cyc_per_mb","c_cycles","mem_mb"]:
        if col in tasks.columns:
            _require((tasks[col] >= 0).all(), f"[{dataset_name}] tasks.{col} has negative values", errors)

    # 3.5 Deadline coherence
    if "has_deadline" in tasks.columns and "deadline_s" in tasks.columns:
        bad_deadline = tasks[(tasks["has_deadline"] == 1) & ((tasks["deadline_s"].isna()) | (tasks["deadline_s"] <= 0))]
        _require(len(bad_deadline) == 0, f"[{dataset_name}] tasks with deadline have invalid deadline_s", errors)

    # 3.6 Episode-level parameters
    _require(episodes["Delta"].nunique() == 1, f"[{dataset_name}] multiple Delta values in episodes", errors)
    _require(episodes["T_slots"].nunique() == 1, f"[{dataset_name}] multiple T_slots in episodes", errors)

    return errors

# ---------- Topology-level validation ----------
def validate_one_topology(topology_name: str, topo_entry: dict) -> list:
    """
    Validate a single topology pack (topology.json + meta + connection_matrix.csv).
    topo_entry expected keys:
      - "topology_data": dict (loaded JSON)
      - "meta_data": dict (loaded JSON)
      - "connection_matrix": DataFrame (loaded)
    """
    errors = []
    topo = topo_entry.get("topology_data")
    meta = topo_entry.get("meta_data")
    Mdf  = topo_entry.get("connection_matrix")

    # Presence checks
    _require(isinstance(topo, dict), f"[{topology_name}] topology_data missing or not a dict", errors)
    _require(isinstance(meta, dict), f"[{topology_name}] meta_data missing or not a dict", errors)
    _require(isinstance(Mdf,  pd.DataFrame), f"[{topology_name}] connection_matrix CSV missing or not a DataFrame", errors)

    if errors:
        return errors

    # Required keys in topology.json
    req_keys = [
        "number_of_servers","private_cpu_capacities","public_cpu_capacities",
        "cloud_computational_capacity","connection_matrix","time_step"
    ]
    for k in req_keys:
        _require(k in topo, f"[{topology_name}] topology.json missing key: {k}", errors)

    if errors:
        return errors

    K = int(topo["number_of_servers"])
    # Length checks for capacities
    _require(len(topo["private_cpu_capacities"]) == K, f"[{topology_name}] private_cpu_capacities length != K", errors)
    _require(len(topo["public_cpu_capacities"])  == K, f"[{topology_name}] public_cpu_capacities length != K", errors)

    # Connection matrix shape (from JSON and CSV)
    Mjson = topo["connection_matrix"]
    _require(isinstance(Mjson, list) and len(Mjson) == K and len(Mjson[0]) == K+1,
             f"[{topology_name}] connection_matrix in JSON must be K x (K+1)", errors)
    _require(Mdf.shape == (K, K+1), f"[{topology_name}] connection_matrix.csv shape must be K x (K+1)", errors)

    # Positive vertical capacities (MEC->Cloud)
    vert_csv = Mdf.iloc[:, K]
    _require((vert_csv > 0).all(), f"[{topology_name}] MEC->Cloud capacities must be > 0", errors)

    # Non-negative horizontal capacities
    horiz_csv = Mdf.iloc[:, :K]
    _require((horiz_csv.values >= 0).all(), f"[{topology_name}] MEC<->MEC capacities contain negatives", errors)

    # Per-slot note: time_step exists
    _require("time_step" in topo, f"[{topology_name}] missing time_step", errors)

    return errors

# ---------- Pairwise validation (dataset <-> topology) ----------
def validate_dataset_topology_pair(dataset_name: str, ds: dict, topology_name: str, topo_entry: dict) -> list:
    """
    Validate alignment between one dataset scenario and one topology.
    Ensures Delta == time_step and basic feasibility checks.
    """
    errors = []
    episodes = ds["episodes"]
    topo     = topo_entry["topology_data"]
    K        = int(topo["number_of_servers"])

    # Delta vs time_step
    Delta = float(episodes["Delta"].iloc[0])
    time_step = float(topo["time_step"])
    _require(abs(Delta - time_step) < 1e-9,
             f"[{dataset_name} x {topology_name}] Delta ({Delta}) != time_step ({time_step})", errors)

    # Basic feasibility: at least one capacity positive
    priv = topo["private_cpu_capacities"]
    pub  = topo["public_cpu_capacities"]
    cloud = topo["cloud_computational_capacity"]
    _require(all(x >= 0 for x in priv) and all(x >= 0 for x in pub) and cloud >= 0,
             f"[{dataset_name} x {topology_name}] negative compute capacities detected", errors)

    # Optional sanity: arrivals span within T_slots
    T_slots = int(episodes["T_slots"].iloc[0])
    tasks = ds["tasks"]
    _require(tasks["t_arrival_slot"].max() <= T_slots - 1,
             f"[{dataset_name} x {topology_name}] t_arrival_slot exceeds T_slots-1", errors)

    # Optional: map agents to MEC (simple modulo mapping) to check index bounds
    N_agents = int(episodes["N_agents"].iloc[0])
    mapped = [(aid % K) for aid in range(N_agents)]
    _require(all(0 <= m < K for m in mapped),
             f"[{dataset_name} x {topology_name}] agent->MEC mapping out of bounds", errors)

    return errors

# ---------- Orchestrator over ALL datasets and ALL topologies ----------
def validate_everything(datasets: dict, topologies: dict) -> dict:
    """
    Iterate through all dataset scenarios and all topologies, run validations,
    and return a structured report.
    'datasets' example shape:
        {
          "light": {"episodes": df, "agents": df, "arrivals": df, "tasks": df},
          "moderate": {...},
          "heavy": {...}
        }
    'topologies' example shape from your loader:
        {
          "full_mesh": {"topology_data": dict, "meta_data": dict, "connection_matrix": df, "paths": {...}},
          "clustered": {...},
          "sparse_ring": {...}
        }
    """
    report = {"datasets": {}, "topologies": {}, "pairs": {}}

    # Validate each dataset
    for dname, dpack in datasets.items():
        errs = validate_one_dataset(dname, dpack)
        report["datasets"][dname] = {"ok": len(errs) == 0, "errors": errs}

    # Validate each topology
    for tname, tpack in topologies.items():
        errs = validate_one_topology(tname, tpack)
        report["topologies"][tname] = {"ok": len(errs) == 0, "errors": errs}

    # Pairwise validation only if both sides are OK
    for dname, dres in report["datasets"].items():
        for tname, tres in report["topologies"].items():
            key = f"{dname}__{tname}"
            if dres["ok"] and tres["ok"]:
                errs = validate_dataset_topology_pair(dname, datasets[dname], tname, topologies[tname])
                report["pairs"][key] = {"ok": len(errs) == 0, "errors": errs}
            else:
                # Skip pair if either side invalid
                report["pairs"][key] = {"ok": False, "errors": ["Skipped due to upstream invalid dataset/topology."]}

    return report

# ---------- Pretty printer ----------
def print_validation_report(report: dict):
    # Datasets
    print("=== DATASETS ===")
    for name, info in report["datasets"].items():
        status = "OK" if info["ok"] else "FAIL"
        print(f"[{status}] {name}")
        for e in info["errors"]:
            print(f"  - {e}")

    # Topologies
    print("\n=== TOPOLOGIES ===")
    for name, info in report["topologies"].items():
        status = "OK" if info["ok"] else "FAIL"
        print(f"[{status}] {name}")
        for e in info["errors"]:
            print(f"  - {e}")

    # Pairs
    print("\n=== DATASET × TOPOLOGY PAIRS ===")
    for key, info in report["pairs"].items():
        status = "OK" if info["ok"] else "FAIL"
        print(f"[{status}] {key}")
        for e in info["errors"]:
            print(f"  - {e}")


In [64]:
report = validate_everything(datasets, topologies)
print_validation_report(report)

# If you need programmatic checks:
all_ok = all(v["ok"] for v in report["datasets"].values()) \
         and all(v["ok"] for v in report["topologies"].values()) \
         and all(v["ok"] for v in report["pairs"].values())
if not all_ok:
    raise RuntimeError("Validation failed. See printed report for details.")


=== DATASETS ===
[OK] heavy
[OK] light
[OK] moderate

=== TOPOLOGIES ===
[OK] clustered
[OK] full_mesh
[OK] sparse_ring

=== DATASET × TOPOLOGY PAIRS ===
[OK] heavy__clustered
[OK] heavy__full_mesh
[OK] heavy__sparse_ring
[OK] light__clustered
[OK] light__full_mesh
[OK] light__sparse_ring
[OK] moderate__clustered
[OK] moderate__full_mesh
[OK] moderate__sparse_ring


<h3> 3. Units Alignment </h3>

In this section, we align units for all dataset scenarios (light / moderate / heavy) and all topologies (full_mesh / clustered / sparse_ring), and run several consistency checks. 


- In datasets: we take Delta from episodes.csv, create deadline_slots for tasks, and add helper per-slot columns for agents (f_local_slot)

- In topologies: capacities are already per-slot (you multiplied by Δ in the generator), so we only verify that time_step == Delta and capacities are positive.

In [67]:
# ===== Imports =====
from typing import Dict, Any, Tuple
import numpy as np
import pandas as pd
import math

# ===== Helpers: safe getters =====
def _get_delta(episodes_df: pd.DataFrame) -> float:
    # Expect a single Delta value in episodes; take the first row
    if "Delta" not in episodes_df.columns:
        raise ValueError("episodes.csv must contain a 'Delta' column.")
    return float(episodes_df["Delta"].iloc[0])

def _ensure_numeric_positive(name: str, arr: np.ndarray):
    # Basic sanity: no negative and finite values for capacities/links
    if not np.isfinite(arr).all():
        raise ValueError(f"{name} contains non-finite values.")
    if (arr < 0).any():
        raise ValueError(f"{name} contains negative values.")

# ===== Alignment: per-dataset =====
def align_units_for_dataset(dataset: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    """
    Given one dataset dict {"episodes","agents","arrivals","tasks"},
    return a copy with aligned/derived columns (per-slot helpers).
    """
    episodes = dataset["episodes"].copy()
    agents   = dataset["agents"].copy()
    arrivals = dataset["arrivals"].copy()
    tasks    = dataset["tasks"].copy()

    Delta = _get_delta(episodes)

    # --- Agents: add per-slot compute capacity helper (cycles/slot) ---
    # f_local is Hz (cycles per second) in generator; we add f_local_slot = f_local * Delta
    if "f_local" in agents.columns:
        agents["f_local_slot"] = agents["f_local"].astype(float) * Delta
    else:
        raise ValueError("agents.csv must contain 'f_local'.")

    # Memory is MB (not per second), so we keep m_local as-is. Add safe dtype.
    if "m_local" in agents.columns:
        agents["m_local"] = agents["m_local"].astype(float)

    # --- Tasks: add deadline_slots; keep arrival slot as integer ---
    if "t_arrival_slot" not in tasks.columns:
        raise ValueError("tasks.csv must contain 't_arrival_slot'.")
    tasks["t_arrival_slot"] = tasks["t_arrival_slot"].astype(int)

    # Build deadline_slots = ceil(deadline_s / Delta) when has_deadline == 1, else NaN
    if "has_deadline" in tasks.columns and "deadline_s" in tasks.columns:
        def _to_deadline_slots(row):
            if int(row["has_deadline"]) == 1 and np.isfinite(row["deadline_s"]):
                return int(math.ceil(float(row["deadline_s"]) / Delta))
            return np.nan
        tasks["deadline_slots"] = tasks.apply(_to_deadline_slots, axis=1)

    # Ensure key numeric task fields are floats
    for col in ["b_mb", "rho_cyc_per_mb", "c_cycles", "mem_mb"]:
        if col in tasks.columns:
            tasks[col] = tasks[col].astype(float)

    return {
        "episodes": episodes,
        "agents":   agents,
        "arrivals": arrivals,
        "tasks":    tasks,
    }

# ===== Verification: per-topology against a target Delta =====
def verify_topology_units(topology: Dict[str, Any], target_Delta: float) -> Tuple[bool, str]:
    """
    Ensure topology capacities are per-slot and consistent with dataset Delta:
    - time_step == target_Delta
    - shapes are valid (K x (K+1))
    - capacities non-negative
    Returns (ok, message).
    """
    # time_step check
    ts = float(topology.get("time_step", -1.0))
    if not np.isclose(ts, target_Delta, atol=1e-9):
        return (False, f"time_step mismatch (topology={ts}, dataset Delta={target_Delta})")

    # K and lists
    K = int(topology.get("number_of_servers", -1))
    priv = np.array(topology.get("private_cpu_capacities", []), dtype=float)
    pub  = np.array(topology.get("public_cpu_capacities", []), dtype=float)
    cloud = float(topology.get("cloud_computational_capacity", -1.0))
    M = np.array(topology.get("connection_matrix", []), dtype=float)

    if K <= 0:
        return (False, "Invalid 'number_of_servers' (K<=0).")
    if priv.shape[0] != K or pub.shape[0] != K:
        return (False, "private/public capacities must have length K.")
    if M.shape != (K, K+1):
        return (False, f"connection_matrix shape must be (K, K+1), got {M.shape}.")

    # Non-negative checks
    _ensure_numeric_positive("private_cpu_capacities", priv)
    _ensure_numeric_positive("public_cpu_capacities",  pub)
    if not np.isfinite(cloud) or cloud < 0:
        return (False, "cloud_computational_capacity must be non-negative and finite.")
    _ensure_numeric_positive("connection_matrix", M)

    return (True, "topology verified (per-slot, consistent).")

# ===== Batch alignment for ALL datasets & ALL topologies =====
def align_all_units(datasets_by_name: Dict[str, Dict[str, pd.DataFrame]],
                    topologies_by_name: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    """
    - Align each dataset variant independently.
    - Verify each topology variant against each dataset's Delta (they should match).
    - Return a structured dict containing aligned datasets and topology checks.
    """
    out = {
        "datasets_aligned": {},   # scenario_name -> aligned dataset dict
        "topology_checks":  {},   # topology_name -> per-dataset Delta check results
    }

    # Align datasets
    for scen, ds in datasets_by_name.items():
        try:
            out["datasets"][scen] = align_units_for_dataset(ds)
        except Exception as e:
            raise RuntimeError(f"[{scen}] dataset alignment failed: {e}") from e

    # Verify topologies against each dataset's Delta
    for topo_name, topo_bundle in topologies_by_name.items():
        # Expect topology_data already loaded into dict
        topo_obj = topo_bundle.get("topology_data", None)
        if not isinstance(topo_obj, dict):
            raise RuntimeError(f"[{topo_name}] 'topology_data' missing or not a dict.")

        per_dataset_results = {}
        for scen, aligned in out["datasets"].items():
            Delta = _get_delta(aligned["episodes"])
            ok, msg = verify_topology_units(topo_obj, Delta)
            per_dataset_results[scen] = {"ok": bool(ok), "message": msg}
        out["topology_checks"][topo_name] = per_dataset_results

    return out

# ===== Pretty printer (optional) =====
def print_alignment_summary(result: Dict[str, Any]):
    # Datasets
    print("=== DATASETS (aligned) ===")
    for scen, ds in result["datasets"].items():
        Delta = _get_delta(ds["episodes"])
        n_tasks = len(ds["tasks"])
        n_agents = len(ds["agents"])
        print(f"[{scen}] Delta={Delta}  tasks={n_tasks}  agents={n_agents}")

    # Topologies
    print("\n=== TOPOLOGIES (checks) ===")
    for topo_name, checks in result["topology_checks"].items():
        print(f"Topology: {topo_name}")
        for scen, r in checks.items():
            flag = "OK" if r["ok"] else "FAIL"
            print(f"  - vs {scen}: {flag}  -> {r['message']}")


In [68]:
# Example driver (after your loading step)
# datasets: { "light": {...}, "moderate": {...}, "heavy": {...} }
# topologies: { "full_mesh": {...}, "clustered": {...}, "sparse_ring": {...} }

result_align = align_all_units(datasets_by_name=datasets, topologies_by_name=topologies)
print_alignment_summary(result_align)

# Access aligned data for a specific scenario:
aligned_light = result_align["datasets_aligned"]["light"]
episodes_light = aligned_light["episodes"]
agents_light   = aligned_light["agents"]   # has f_local_slot
tasks_light    = aligned_light["tasks"]    # has deadline_slots


=== DATASETS (aligned) ===
[heavy] Delta=1.0  tasks=30636  agents=18
[light] Delta=1.0  tasks=2113  agents=18
[moderate] Delta=1.0  tasks=8262  agents=18

=== TOPOLOGIES (checks) ===
Topology: clustered
  - vs heavy: OK  -> topology verified (per-slot, consistent).
  - vs light: OK  -> topology verified (per-slot, consistent).
  - vs moderate: OK  -> topology verified (per-slot, consistent).
Topology: full_mesh
  - vs heavy: OK  -> topology verified (per-slot, consistent).
  - vs light: OK  -> topology verified (per-slot, consistent).
  - vs moderate: OK  -> topology verified (per-slot, consistent).
Topology: sparse_ring
  - vs heavy: OK  -> topology verified (per-slot, consistent).
  - vs light: OK  -> topology verified (per-slot, consistent).
  - vs moderate: OK  -> topology verified (per-slot, consistent).


<h3> 4. Build Scenario–Topology Pairs </h3>

In this step, all datasets are paired with all topologies (Cartesian product). Each pair is checked for matching time parameters, then a basic bundle is created for further enrichment.

In [81]:
from typing import Dict, Any
import numpy as np

def _delta_from_episodes(episodes_df) -> float:
    # Get single Delta value from episodes table
    if "Delta" not in episodes_df.columns:
        raise ValueError("episodes.csv must contain 'Delta'.")
    return float(episodes_df["Delta"].iloc[0])

def _topology_time_step(topo_json: Dict[str, Any]) -> float:
    # Extract topology time_step
    ts = topo_json.get("time_step", None)
    if ts is None:
        raise ValueError("topology.json must contain 'time_step'.")
    return float(ts)

def build_topology_scenario_pairs(
    datasets: Dict[str, Dict[str, Any]],
    topologies: Dict[str, Dict[str, Any]],
    strict_delta_match: bool = True
) -> Dict[str, Dict[str, Any]]:
    """
    Build a nested dict organized as:
      pairs_by_topology[topology_name][scenario_name] = {
        'scenario': <scenario_name>,
        'topology': <topology_name>,
        'Delta': <float>,
        'K': <int>,
        'dataset': {episodes, agents, arrivals, tasks},
        'topology_data': <dict>,
        'topology_meta_data': <dict or None>,
        'connection_matrix_df': <pd.DataFrame>,  # shape (K, K+1)
        'checks': {'delta_match': bool, 'message': str}
      }

    If strict_delta_match=True, a mismatch between dataset Delta and topology time_step raises an error.
    """
    pairs_by_topology: Dict[str, Dict[str, Any]] = {}

    # Iterate over all topologies first (topology-centric)
    for topo_name, topo_bundle in topologies.items():
        topo_data = topo_bundle.get("topology_data", None)
        meta_data = topo_bundle.get("meta_data", None)
        cm_df     = topo_bundle.get("connection_matrix", None)

        if not isinstance(topo_data, dict):
            raise ValueError(f"[{topo_name}] topology_data missing or not a dict.")
        if cm_df is None:
            raise ValueError(f"[{topo_name}] connection_matrix DataFrame is missing.")

        # Extract K for shape checks
        K = int(topo_data.get("number_of_servers", -1))
        if K <= 0:
            raise ValueError(f"[{topo_name}] invalid 'number_of_servers' in topology.json")

        # Validate connection_matrix dimension K x (K+1)
        if not (cm_df.shape[0] == K and cm_df.shape[1] == K + 1):
            raise ValueError(
                f"[{topo_name}] connection_matrix shape must be (K, K+1); got {cm_df.shape}"
            )

        # Prepare nested dict for this topology
        if topo_name not in pairs_by_topology:
            pairs_by_topology[topo_name] = {}

        # Compare with every scenario
        topo_ts = _topology_time_step(topo_data)
        for scen_name, ds in datasets.items():
            ds_Delta = _delta_from_episodes(ds["episodes"])
            delta_ok = bool(np.isclose(ds_Delta, topo_ts, atol=1e-12))
            msg = "OK" if delta_ok else (
                f"time_step mismatch (dataset Delta={ds_Delta}, topology time_step={topo_ts})"
            )
            if (not delta_ok) and strict_delta_match:
                raise ValueError(f"[{topo_name} × {scen_name}] {msg}")

            # Store a topology→scenario bundle
            pairs_by_topology[topo_name][scen_name] = {
                "scenario": scen_name,
                "topology": topo_name,
                "Delta": ds_Delta,
                "K": K,
                "dataset": ds,
                "topology_data": topo_data,
                "topology_meta_data": meta_data,
                "connection_matrix_df": cm_df,
                "checks": {"delta_match": delta_ok, "message": msg}
            }

    return pairs_by_topology

def print_pairs_summary_topology_first(pairs_by_topology: Dict[str, Dict[str, Any]]) -> None:
    # Pretty-print a compact summary in topology→scenario order
    print("=== TOPOLOGY PAIRS × SCENARIO ===")
    for topo_name, scen_map in pairs_by_topology.items():
        print(f"[TOPOLOGY] {topo_name}")
        for scen_name, bundle in scen_map.items():
            flag  = "OK" if bundle["checks"]["delta_match"] else "FAIL"
            K     = bundle["K"]
            Delta = bundle["Delta"]
            msg   = bundle["checks"]["message"]
            print(f"  - [{flag}] {topo_name} :: {scen_name}  |  K={K}  Delta={Delta}  -> {msg}")


In [82]:
# Assume you already have:
# datasets: Dict[str, Dict[str, Any]]  # e.g., {"light": {...}, "moderate": {...}, "heavy": {...}}
# topologies:       Dict[str, Dict[str, Any]]  # e.g., {"full_mesh": {...}, "clustered": {...}, "sparse_ring": {...}}

pairs_by_topology = build_topology_scenario_pairs(
    datasets=datasets,
    topologies=topologies,
    strict_delta_match=True  # set False if you want to allow mismatches but mark them as FAIL
)

print_pairs_summary_topology_first(pairs_by_topology)

# Access pattern:
# pairs_by_topology["full_mesh"]["light"]["dataset"]["tasks"]  -> tasks DataFrame for (full_mesh, light)
# pairs_by_topology["clustered"]["heavy"]["connection_matrix_df"] -> K x (K+1) matrix for that topology


=== TOPOLOGY PAIRS × SCENARIO ===
[TOPOLOGY] clustered
  - [OK] clustered :: heavy  |  K=18  Delta=1.0  -> OK
  - [OK] clustered :: light  |  K=18  Delta=1.0  -> OK
  - [OK] clustered :: moderate  |  K=18  Delta=1.0  -> OK
[TOPOLOGY] full_mesh
  - [OK] full_mesh :: heavy  |  K=18  Delta=1.0  -> OK
  - [OK] full_mesh :: light  |  K=18  Delta=1.0  -> OK
  - [OK] full_mesh :: moderate  |  K=18  Delta=1.0  -> OK
[TOPOLOGY] sparse_ring
  - [OK] sparse_ring :: heavy  |  K=18  Delta=1.0  -> OK
  - [OK] sparse_ring :: light  |  K=18  Delta=1.0  -> OK
  - [OK] sparse_ring :: moderate  |  K=18  Delta=1.0  -> OK


<h3> 5. Agent→MEC mapping (for all pairs) </h3>